<a href="https://colab.research.google.com/github/thiagolermen/ml-course/blob/main/src/2-multivariate-linear-regression/PyTorch-multivariate-linea-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multivariate Linear Regression - PyTorch

We're gonna predict the price that a house will sell for. The difference this time around is we have more than one dependent variable. We're given both the size of the house in square feet, and the number of bedrooms in the house

## Imports

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn

## Dataset

In [6]:
path = 'https://raw.githubusercontent.com/thiagolermen/ml-course/main/data/ex1data2.txt?token=AL353PBOIXU364U56BAPW6TAXPWLS'
data = pd.read_csv(path, header=None, names=['Size', 'Bedrooms', 'Price'])
data.head()

,Size,Bedrooms,Price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


### Data normalization

In [7]:
# We put this values normally distributed
data = (data - data.mean())/data.std()
data.head()

,Size,Bedrooms,Price
0,0.130010,-0.223675,0.475747
1,-0.504190,-0.223675,-0.084074
2,0.502476,-0.223675,0.228626
3,-0.735723,-1.537767,-0.867025
4,1.257476,1.090417,1.595389


### Data preprocessing

In [8]:
# set X (training data) and y (target variable)
X = data.iloc[:,0:2]
y = data.iloc[:,2:]

# convert to matrices and initialize theta
X = np.matrix(X.values, dtype="float32")
y = np.matrix(y.values, dtype="float32")

# convert to torch tensor
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [9]:
X.shape, y.shape

(torch.Size([47, 2]), torch.Size([47, 1]))

Create TensorDataset

In [11]:
from torch.utils.data import TensorDataset
# create a TensorDataset
train_ds = TensorDataset(X, y)
train_ds[0:3]

(tensor([[ 0.1300, -0.2237],
         [-0.5042, -0.2237],
         [ 0.5025, -0.2237]]), tensor([[ 0.4757],
         [-0.0841],
         [ 0.2286]]))

Create DataLoader

In [12]:
from torch.utils.data import DataLoader
# create DataLoader
batch_size = 4
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

Prints the first batch

In [14]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[-0.2940, -0.2237],
        [ 0.7126,  1.0904],
        [ 1.2575,  1.0904],
        [ 1.2965,  1.0904]])
tensor([[-0.6999],
        [-0.2112],
        [ 1.5954],
        [ 2.0680]])


## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [15]:
model = nn.Linear(2,1)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2417, -0.3701]], requires_grad=True)
Parameter containing:
tensor([0.4146], requires_grad=True)


In [16]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2417, -0.3701]], requires_grad=True), Parameter containing:
 tensor([0.4146], requires_grad=True)]

Predictions

In [18]:
preds = model(X)
preds

tensor([[ 4.6596e-01],
        [ 6.1923e-01],
        [ 3.7594e-01],
        [ 1.1616e+00],
        [-2.9290e-01],
        [ 1.5778e-02],
        [ 6.3930e-01],
        [ 6.7184e-01],
        [ 6.8614e-01],
        [ 6.5147e-01],
        [ 2.9463e-02],
        [ 4.9758e-01],
        [ 5.3104e-01],
        [-1.2287e+00],
        [ 7.2020e-01],
        [-8.0018e-02],
        [ 1.1907e+00],
        [ 7.2993e-01],
        [-1.7399e-01],
        [-3.0233e-01],
        [ 5.6844e-01],
        [ 1.0180e+00],
        [ 6.1801e-01],
        [ 2.2773e-02],
        [-7.7195e-02],
        [ 7.7129e-01],
        [ 6.6241e-01],
        [ 3.3762e-01],
        [ 4.3676e-01],
        [ 3.0386e-01],
        [ 1.0329e+00],
        [ 1.7744e+00],
        [-9.4828e-04],
        [ 1.5180e-01],
        [ 6.8694e-02],
        [ 6.6880e-01],
        [ 7.2902e-01],
        [-2.8927e-02],
        [-6.6240e-01],
        [-3.8050e-02],
        [ 1.0861e+00],
        [ 4.2520e-01],
        [-1.6122e-01],
        [ 7

## Loss function

In [19]:
import torch.nn.functional as F
# define loss function
loss_fn = F.mse_loss

In [20]:
loss = loss_fn(model(X), y)
print(loss)

tensor(2.1648, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer ```optim.SGD```. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [22]:
# define optimizer

opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model

In [25]:
def fit(num_epochs, model, loss_fn, opt, train_dl):

  for epoch in range(num_epochs):

    # Train with batches
    for xb, yb in train_dl:

      # 1: generate predictions
      pred = model(xb)

      # 2: calculate loss
      loss = loss_fn(pred, yb)

      # 3: compute gradients
      loss.backward()

      # 4: update parameters using gradients
      opt.step()

      # 5: reset the gradients to zero
      opt.zero_grad()

    if (epoch+1) % 10 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [26]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 1.3918
Epoch [20/100], Loss: 1.1094
Epoch [30/100], Loss: 3.3361
Epoch [40/100], Loss: 0.9726
Epoch [50/100], Loss: 0.2665
Epoch [60/100], Loss: 1.8022
Epoch [70/100], Loss: 1.5538
Epoch [80/100], Loss: 2.5872
Epoch [90/100], Loss: 2.6365
Epoch [100/100], Loss: 0.4371


## Predictions

In [27]:
# generate predictions

preds = model(X)
preds

tensor([[ 4.4464e-01],
        [ 5.6036e-01],
        [ 3.7668e-01],
        [ 1.0329e+00],
        [-1.9142e-01],
        [ 4.1614e-02],
        [ 5.7551e-01],
        [ 6.0007e-01],
        [ 6.1087e-01],
        [ 5.8469e-01],
        [ 5.1946e-02],
        [ 4.6852e-01],
        [ 4.9377e-01],
        [-9.6111e-01],
        [ 6.3658e-01],
        [-3.0708e-02],
        [ 1.0550e+00],
        [ 6.4393e-01],
        [-1.0165e-01],
        [-1.9854e-01],
        [ 5.2201e-01],
        [ 9.2458e-01],
        [ 5.5944e-01],
        [ 4.6895e-02],
        [ 3.4583e-02],
        [ 6.7515e-01],
        [ 5.9296e-01],
        [ 3.4775e-01],
        [ 4.2260e-01],
        [ 3.2227e-01],
        [ 9.3583e-01],
        [ 1.5588e+00],
        [ 2.8986e-02],
        [ 2.0747e-01],
        [ 8.1563e-02],
        [ 5.9778e-01],
        [ 6.4324e-01],
        [ 7.8636e-03],
        [-4.7038e-01],
        [ 9.7579e-04],
        [ 9.7601e-01],
        [ 4.1387e-01],
        [-9.2010e-02],
        [ 6

In [29]:
# compare with targets
y

tensor([[ 0.4757],
        [-0.0841],
        [ 0.2286],
        [-0.8670],
        [ 1.5954],
        [-0.3240],
        [-0.2040],
        [-1.1309],
        [-1.0270],
        [-0.7831],
        [-0.8031],
        [ 0.0527],
        [-0.0833],
        [ 2.8750],
        [-0.6439],
        [ 0.8756],
        [-0.3240],
        [-1.1237],
        [ 1.2763],
        [ 2.0680],
        [-0.6999],
        [-0.6831],
        [-0.7799],
        [-0.6439],
        [ 1.8673],
        [-0.7239],
        [ 0.9924],
        [ 1.0284],
        [ 1.0764],
        [-0.3240],
        [ 0.0759],
        [-1.3637],
        [-0.2040],
        [ 1.9153],
        [-0.4360],
        [-0.7239],
        [-0.8838],
        [ 0.0367],
        [ 1.6682],
        [-0.4272],
        [ 0.2246],
        [-0.0841],
        [-0.2112],
        [-0.3312],
        [-1.2837],
        [-0.3240],
        [-0.8070]])